In [98]:
import pandas as pd

import re

import os

import operator

import pymorphy2

import nltk

from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«"
full_punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–,»«'

In [99]:
df_sence = pd.read_csv("./for calculation/mult_sence.csv")
sence_dict = Series(df_sence.count_rank.values,index=df_sence.word).to_dict()
df_sence.head()

,index,word,count_rank
0,0,а,2
1,1,аберрация,2
2,2,абстракция,1
3,3,аван,1
4,4,авангард,2


In [20]:
def low(word):
    return word.lower()
low("Раисся")

'раисся'

In [21]:
df_freq = pd.read_csv("./for calculation/freq_lables.csv")
df_freq['Lemma'] = df_freq['Lemma'].apply(low)
freq_pos_dict = Series(df_freq.PoS.values,index=df_freq.Lemma).to_dict()
freq_dict = Series(df_freq.freq_lable.values,index=df_freq.Lemma).to_dict()
#df_freq = pd.read_csv("freq_lables.csv")
df_freq.head()

,Unnamed: 0,Lemma,PoS,freq_lable
0,0,и,conj,0
1,1,в,pr,0
2,2,не,part,0
3,3,на,pr,0
4,4,я,spro,0


In [26]:
freq_dict['по']

2

In [6]:
df_eng = pd.read_csv("./for calculation/eng_words.csv", sep = ' ')
eng_words = set(df_eng['orth'])
df_eng.head()

,orth,phon,sense,pos,gender,asp,dic_name,usg,etym_lang
0,аврал,АВР'АЛ,Общая работа на судне,сущ.,м.,None,Толковый словарь Кузнецова,None,англ.
1,автокар,АВТОК'АР,автотележка.,сущ.,м.,None,Толковый словарь Кузнецова,None,англ.
2,адаптер,АД'АПТЕР,= Звукосниматель.,сущ.,м.,None,Толковый словарь Кузнецова,None,англ.
3,айсберг,'АЙСБЕРГ,"о ком-л. не понятом, не разгаданном окружающи...",сущ.,м.,None,Толковый словарь Кузнецова,None,англ.
4,акваланг,АКВАЛ'АНГ,"Аппарат, состоящий из баллонов со сжатым возд...",сущ.,м.,None,Толковый словарь Кузнецова,None,англ.


In [7]:
abs_suff={'аж',
 'есть',
 'ие',
 'изм',
 'изна',
 'ина',
 'ость',
 'ота',
 'ствo',
 'ция',
 'чина',
 'щина',
 'ёж','еж'}

In [8]:
def create_lex_vector(lexema, POS):
    #если слова нет в часотном словаре, значит оно скорее всего довольно редкое и признаем его относящ к категории 2
    #если слово не заимствовано то признаем сложным 1 
    sence_value, freq_value, eng_value, abstract= 0,2,1,0
    if(lexema in sence_dict):
        #print(lexema, "found")
        sence_value = sence_dict[lexema]
        
        
    if(lexema in freq_dict):
        freq_value = freq_dict[lexema]
    
    if(lexema in eng_words):
        eng_value = 0
    
    if (POS == "NOUN"):
        for suff in abs_suff:
            if (lexema.endswith(suff)):
                abstract = 1

    return [sence_value, freq_value, eng_value, abstract]  

In [10]:
morph = pymorphy2.MorphAnalyzer()
morph.parse("делать")[0].tag.POS

'INFN'

In [11]:
test_line = "Утром у здания Минкомсвязи разверзлась белизна на Тверской улице произошла драка. В ней погиб чемпион России по рукопашному бою Геннадий Павлов"

In [80]:
import itertools

iterables = [ [0,1,2], [0,1,2],[0,1],[0,1] ]
poss_var = []
for t in itertools.product(*iterables):
    #print (t)
    poss_var.append(str(t)[1:-1])
print(len(poss_var))
vocab_features_dict = OrderedDict.fromkeys(poss_var, 0)

36


In [85]:
vocab_features_dict['0, 2, 1, 0']

0

In [102]:
def claculate_line_vocab_vector(text,morph, vocab_features_dict):
    """
    составляем словарь лемматизированных существительных и словарь всех лемм 
    """
    #lex_vectors = []
    lex_dict_check = {}
    points_dict_for_database = {}
    words = text.split()
    for word in words:
        parsed_word = morph.parse(word)[0]
        lemma = parsed_word.normal_form      
        pos = parsed_word.tag.POS
        #print(lemma, pos)
        vector = create_lex_vector(lemma, pos)
        vector_id = str(vector)[1:-1]
        
        if(vector_id in vocab_features_dict):
          vocab_features_dict [vector_id] += 1
        else:
            print("UNKNOWN VALUES {}".format(vector_id))
          #points_dict_for_database [ str(vector)[1:-1]] = 1
        
        
        #lex_vectors.append(vector)
        lex_dict_check[lemma] = vector
    #print(lex_dict_check)

claculate_line_vocab_vector(test_line, morph, vocab_features_dict)

In [34]:
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    clean_sentence = []
    #print(sentences)
    for sentence in sentences:
        words = sentence.split()
        clean_text= ''
        for word in words:
            clean_word = ''
            for char in word:
                if char != " " and char not in full_punctuation:
                    clean_word += char.lower()
            clean_text += clean_word + ' '
        clean_text = re.sub(' +', ' ', clean_text)
        clean_text = clean_text.strip()
        clean_sentence.append(clean_text)
    return clean_sentence   

def clean_file(file):
    preprocessed_text = []
    lines = 0
    with open (file, "r", encoding = "utf-8") as file:
        all_lines = file.readlines()
        
        for line in all_lines:
            t = preprocess_text(line)
            #print(t)
            preprocessed_text.extend(t)
            lines += 1

    return preprocessed_text

In [40]:
clean_txt = clean_file("text_8.txt")

In [44]:
def calculate_lix_from_list_of_sentences(processed_text_sentences):
        sentences_count = len(processed_text_sentences)
        words_count = sum([len(line.split(' ')) for line in processed_text_sentences])
        long_words_count = 0 #more than 6
        for line in processed_text_sentences:
            for word in line.split():
                if len(word) > 6:
                    long_words_count += 1
        lix = words_count/ sentences_count + (long_words_count * 100) / words_count
        print(lix)
        
calculate_lix_from_list_of_sentences(clean_txt)

39.771679197994985


In [47]:
def lemmatize_line(text,morph):
    lemm_split_text = []
    words = text.split()
    for word in words:
        parsed_word = morph.parse(word)[0]
        lemma = parsed_word.normal_form      
        lemm_split_text.append(lemma)
    return ' '.join(lemm_split_text)

def lemmatize_text_from_sent_list(sent_list):
    lemm_text_lines = []
    morph = pymorphy2.MorphAnalyzer()
    for line in sent_list:
        lemm_line = lemmatize_line(line, morph)
        lemm_text_lines.append(lemm_line)
    return lemm_text_lines
lemm_text = lemmatize_text_from_sent_list(clean_txt)

In [49]:
def calculate_type_token_ratio(lemm_text_sentences):
      all_words = []
      for sentence in lemm_text_sentences:
          words = sentence.split()
          for word in words:
              all_words.append(word)

      unqie_words = set(all_words)
      types = len(unqie_words)
      tokens = len (all_words)

      return types/tokens
calculate_type_token_ratio (lemm_text)

0.5338345864661654

In [66]:
from itertools import combinations
from collections import OrderedDict

In [59]:
for j, k in combinations([0,1,2], 2):
    comb = [j,k]
    print(comb)

[0, 1]
[0, 2]
[1, 2]


In [95]:
iterables = [ [0,1,2], [0,1,2],[0,1],[0,1] ]
poss_var = []
for t in itertools.product(*iterables):
    #print (t)
    poss_var.append(str(t)[1:-1])
#print(len(poss_var))
vocab_features_dict_for_calc = OrderedDict.fromkeys(poss_var, 0)

In [122]:
vocvocab_features_dict_for_calcab_features_dict_for_calc = OrderedDict.fromkeys(poss_var, 0)
vocab_features_dict_for_calc.items()[0]

TypeError: 'odict_items' object does not support indexing

In [136]:
vocab_features_dict_for_calc = OrderedDict.fromkeys(poss_var, 0)
def calculate_vocab_feaures_from_sent_list(lemm_sent, vocab_features_dict):
    
    for sent in lemm_sent:
        claculate_line_vocab_vector(sent,morph, vocab_features_dict)
    #print(vocab_features_dict.values())    
    all_vals = vocab_features_dict.values()

    for ind in range(len(vocab_features_dict)):
        el = list(vocab_features_dict.keys())[ind]
        vocab_features_dict[el] /= sum(all_vals)
calculate_vocab_feaures_from_sent_list(lemm_text, vocab_features_dict_for_calc)

vocab_features_dict

OrderedDict([('0, 0, 0, 0', 0.0),
             ('0, 0, 0, 1', 0.0),
             ('0, 0, 1, 0', 0.01),
             ('0, 0, 1, 1', 0.01),
             ('0, 1, 0, 0', 0.01),
             ('0, 1, 0, 1', 0.0),
             ('0, 1, 1, 0', 0.01),
             ('0, 1, 1, 1', 0.0),
             ('0, 2, 0, 0', 0.0),
             ('0, 2, 0, 1', 0.0),
             ('0, 2, 1, 0', 0.07),
             ('0, 2, 1, 1', 0.01),
             ('1, 0, 0, 0', 0.0),
             ('1, 0, 0, 1', 0.0),
             ('1, 0, 1, 0', 0.02),
             ('1, 0, 1, 1', 0.0),
             ('1, 1, 0, 0', 0.0),
             ('1, 1, 0, 1', 0.0),
             ('1, 1, 1, 0', 0.01),
             ('1, 1, 1, 1', 0.0),
             ('1, 2, 0, 0', 0.0),
             ('1, 2, 0, 1', 0.0),
             ('1, 2, 1, 0', 0.0),
             ('1, 2, 1, 1', 0.0),
             ('2, 0, 0, 0', 0.0),
             ('2, 0, 0, 1', 0.0),
             ('2, 0, 1, 0', 0.02),
             ('2, 0, 1, 1', 0.0),
             ('2, 1, 0, 0', 0.01),
    